![](https://www.zorgkaartnederland.nl/bundles/zkncontent/images/elements/logo_zkn.svg)

In [1]:
import zorgkaart

In [2]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# 1. Organisatietypen

In [3]:
types = zorgkaart.get_types()
types.sample(n=7)

,aantal,url
organisatietype,,
Echocentrum,76,https://www.zorgkaartnederland.nl/echocentrum
Centrum voor orthopedie en beweging,66,https://www.zorgkaartnederland.nl/centrum-voor...
Centrum voor bijzondere tandheelkunde,31,https://www.zorgkaartnederland.nl/centrum-voor...
IVF-kliniek,9,https://www.zorgkaartnederland.nl/ivf-kliniek
Brandwondencentrum,3,https://www.zorgkaartnederland.nl/brandwondenc...
Esthetisch-medisch centrum,264,https://www.zorgkaartnederland.nl/esthetisch-m...
Mondhygiënistenpraktijk,658,https://www.zorgkaartnederland.nl/mondhygienis...


# 2. Informatie

### Ophalen informatie

In [4]:
# kies organisatietype
organisatietype = 'Sportmedisch centrum'

In [5]:
# aantal instellingen
print('Voor organisatietype \'{}\' zijn {} instellingen gevonden.'
      .format(organisatietype, types.loc[organisatietype].aantal)
     )

Voor organisatietype 'Sportmedisch centrum' zijn 66 instellingen gevonden.


In [6]:
# ophalen informatie
df = zorgkaart.get_info(organisatietype)

Pagination: 100%|██████████| 4/4 [00:12<00:00,  3.01s/it]


In [7]:
# tonen informatie
df.head()

,id,zorginstelling,plaats,beoordeling,waarderingen,latitude,longitude,zknl_url
0,102463,Sportgeneeskunde Rotterdam,Rotterdam,8.4,43,51.923527,4.538945,https://www.zorgkaartnederland.nl/zorginstelli...
1,10001574,SMA Gelderse Vallei,Ede,10.0,1,52.021933,5.646425,https://www.zorgkaartnederland.nl/zorginstelli...
2,252156,"SMA Gelderse Vallei, locatie Ede",Ede,10.0,1,52.021128,5.645601,https://www.zorgkaartnederland.nl/zorginstelli...
3,10010016,"Sportho, locatie Born",Born,10.0,1,51.036692,5.806560,https://www.zorgkaartnederland.nl/zorginstelli...
4,10009303,Sportmedische Centra Plemper,Amstelveen,10.0,1,52.279252,4.866099,https://www.zorgkaartnederland.nl/zorginstelli...


### Visualiseren informatie

In [8]:
m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 6)

marker_cluster = MarkerCluster().add_to(m)

for row in df.iterrows():
    row_values = row[1]
    location = (row_values.latitude, row_values.longitude)
    instelling = row_values.zorginstelling
    plaats = row_values.plaats
    rating = '{} ({} reviews)'.format(row_values.beoordeling, row_values.waarderingen)
    popup = '<b>{}</b><br><i>{}</i><br>{}'.format(instelling, plaats, rating)
    icon=folium.Icon(color='blue', icon='info-sign')
    marker = folium.Marker(location = location, popup = popup, icon=icon)
    marker.add_to(marker_cluster)

display(m)

# 3. Details

### Ophalen details

In [9]:
# kies organisatietype
organisatietype = 'Zorghotel'

In [10]:
# aantal instellingen
print('Voor organisatietype \'{}\' zijn {} instellingen gevonden.'
      .format(organisatietype, types.loc[organisatietype].aantal)
     )

Voor organisatietype 'Zorghotel' zijn 53 instellingen gevonden.


In [11]:
# ophalen details
df = zorgkaart.get_details(organisatietype)

Zorghotel: 100%|██████████| 53/53 [02:05<00:00,  2.37s/it]


In [12]:
# tonen details
df.head()

,id,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,latitude,longitude,website,zorgkaart_url,wachttijden_url
0,0,Zorghotel De Kim,Rembrandtweg 2,2202 AX,Noordwijk,071-3640730,9.3,41,52.236573,4.424665,http://www.zorghoteldekim.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
1,1,"Swinhove Groep, Revalidatie Ambachtstaete",Plantageweg 3,3333 GZ,Zwijndrecht,078-6255500,8.4,34,51.815084,4.630468,http://www.swinhovegroep.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
2,2,Aafje zorghotel locatie Maasstad,Maasstadweg 2,3079 DZ,Rotterdam,NaN,7.6,248,51.880181,4.536032,http://www.aafje.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3,3,Zorgpension,Antwerpseweg 7,2803 PB,Gouda,NaN,7.4,52,52.020516,4.687316,http://www.zorgpension.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
4,4,Dores Herstelzorg,Mr. B.M. Teldersstraat 5,6842 CT,Arnhem,085-8087030,7.4,47,51.953695,5.870132,https://www.doresherstelzorg.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN


### Visualiseren details

In [13]:
norm_beoordeling = 7

In [14]:
m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 8)
for row in df.iterrows():
    row_values = row[1]
    location = (row_values.latitude, row_values.longitude)
    inst = row_values.zorginstelling
    adr1 = row_values.adres
    adr2 = row_values.postcode + '  ' + row_values.plaats
    tel = row_values.telefoon
    bw = '{} ({} reviews)'.format(row_values.beoordeling, row_values.waarderingen)
    web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
    popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}'.format(inst, adr1, adr2, tel, web, bw)
    if np.isnan(row_values.beoordeling):
        icon=folium.Icon(color='lightgray', icon='question-sign')
    elif row_values.beoordeling >= norm_beoordeling:
        icon=folium.Icon(color='green', icon='ok')
    else:
        icon=folium.Icon(color='lightred', icon='remove')
    marker = folium.Marker(location = location, popup = popup, icon=icon)
    marker.add_to(m)
display(m)

# 4. Wachttijden

### Ophalen wachttijden

In [15]:
# kies organisatietype
organisatietype = 'Ziekenhuis'

In [16]:
# aantal instellingen
print('Voor organisatietype \'{}\' zijn {} instellingen gevonden.'
      .format(organisatietype, types.loc[organisatietype].aantal)
     )

Voor organisatietype 'Ziekenhuis' zijn 299 instellingen gevonden.


In [17]:
# ophalen wachttijden
instelling, wachttijden = zorgkaart.get_wachttijden('Ziekenhuis', reasonable_rate=False)

Wachttijden: 100%|██████████| 204/204 [01:58<00:00,  1.72it/s]


In [18]:
# merge datasets
df = pd.merge(instelling, wachttijden, on='id')\
.drop(['zorginstelling_y'], axis=1)\
.rename(columns = {'zorginstelling_x': 'zorginstelling'})

In [19]:
# tonen wachttijden
df.sample(n=3)

,id,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,latitude,longitude,website,zorgkaart_url,wachttijden_url,specialisme,wachttijd
2420,288,"Reinier de Graaf, locatie Ypenburg",Kiekendiefstraat 17,2496 RP,Den Haag,015-2706800,NaN,0,52.051770,4.385171,http://www.reinierdegraaf.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...,Kindergeneeskunde,9
1174,127,"Treant Zorggroep, Ziekenhuislocatie Scheper",Boermarkeweg 60,7824 AA,Emmen,0591-691911,8.0,124,52.786234,6.913055,http://www.treant.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...,KNO,2
1887,204,ETZ Waalwijk,Kasteellaan 2,5141 BM,Waalwijk,013-2210000,9.1,21,51.685882,5.053028,https://www.etz.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...,Kindergeneeskunde,6


### Visualiseren wachttijden

In [20]:
specialisme = 'Cardiologie'
norm_wachttijd = 4

In [21]:
m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 7)

for row in df[df.specialisme==specialisme].iterrows():
    row_values = row[1]
    location = (row_values.latitude, row_values.longitude)
    inst = row_values.zorginstelling
    adr1 = row_values.adres
    adr2 = row_values.postcode + '  ' + row_values.plaats
    tel = row_values.telefoon
    bw = '{} ({} reviews)'.format(row_values.beoordeling, row_values.waarderingen)
    web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
    wtijd = row_values.wachttijd
    wtext = 'Wachttijd: {} weken'.format(wtijd)
    popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}<br>{}'\
    .format(inst, adr1, adr2, tel, web, bw, wtext)
    if wtijd <= norm_wachttijd:
        icon=folium.Icon(color='green', icon='ok')
    else:
        icon=folium.Icon(color='lightred', icon='remove')
    marker = folium.Marker(location = location, popup = popup, icon=icon)

    marker.add_to(m)

display(m)

### Ophalen geen wachttijden

In [22]:
# filteren dataset
df = instelling[pd.isnull(instelling.wachttijden_url)]

In [23]:
# tonen datset
df.sample(n=3)

,id,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,latitude,longitude,website,zorgkaart_url,wachttijden_url
79,79,Radboudumc,Geert Grooteplein-Zuid 10,6525 GA,Nijmegen,024-3611111,8.3,291,51.823715,5.859974,https://www.radboudumc.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
200,200,Saxenburgh Medisch Centrum,Jan Weitkamplaan 4 A,7772 SE,Hardenberg,0523-276000,9.3,7,52.571252,6.630109,https://www.saxenburgh.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
117,117,"VieCuri, Medisch Centrum voor Noord-Limburg",Postbus 1926,5900 BX,Venlo,077-3205555,8.0,265,51.357645,6.154583,http://www.viecuri.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN


### Visualiseren geen wachttijden

In [24]:
m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 7)

marker_cluster = MarkerCluster().add_to(m)

for row in df.iterrows():
    row_values = row[1]
    location = (row_values.latitude, row_values.longitude)
    inst = row_values.zorginstelling
    adr1 = row_values.adres
    adr2 = row_values.postcode + '  ' + row_values.plaats
    tel = row_values.telefoon
    bw = '{} ({} waarderingen)'.format(row_values.beoordeling, row_values.waarderingen)
    web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
    popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}'\
    .format(inst, adr1, adr2, tel, web, bw)
    icon=folium.Icon(color='lightgray', icon='question-sign')
    marker = folium.Marker(location = location, popup = popup, icon=icon)
    marker.add_to(marker_cluster) 

display(m)